In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from itertools import product

In [2]:
data = pd.read_csv('../data/bybit_ethusdt_20240901-20240930.csv', parse_dates=['Timestamp'])
data.set_index('Timestamp', inplace=True)

In [3]:
# 移動平均線の計算
def calculate_sma(series, period):
    return series.rolling(window=period).mean()

short_ma_period = 5
long_ma_period = 20

data['short_ma'] = calculate_sma(data['Close'], short_ma_period)
data['long_ma'] = calculate_sma(data['Close'], long_ma_period)

# MACDの計算
def calculate_ema(series, period):
    return series.ewm(span=period, adjust=False).mean()

def calculate_macd(series, fast_period, slow_period, signal_period):
    macd_line = calculate_ema(series, fast_period) - calculate_ema(series, slow_period)
    signal_line = macd_line.ewm(span=signal_period, adjust=False).mean()
    return macd_line, signal_line

macd_fast_period = 12
macd_slow_period = 26
macd_signal_period = 9

data['macd_line'], data['signal_line'] = calculate_macd(data['Close'], macd_fast_period, macd_slow_period, macd_signal_period)

# ADXの計算
def calculate_adx(df, period):
    df = df.copy()
    df['Prev_High'] = df['High'].shift(1)
    df['Prev_Low'] = df['Low'].shift(1)
    df['Prev_Close'] = df['Close'].shift(1)
    
    # True Range (TR) の計算
    df['High_Low'] = df['High'] - df['Low']
    df['High_PrevClose'] = abs(df['High'] - df['Prev_Close'])
    df['Low_PrevClose'] = abs(df['Low'] - df['Prev_Close'])
    df['TR'] = df[['High_Low', 'High_PrevClose', 'Low_PrevClose']].max(axis=1)
    
    # Directional Movement (DM) の計算
    df['Plus_DM'] = np.where((df['High'] - df['Prev_High']) > (df['Prev_Low'] - df['Low']),
                             np.maximum(df['High'] - df['Prev_High'], 0), 0)
    df['Minus_DM'] = np.where((df['Prev_Low'] - df['Low']) > (df['High'] - df['Prev_High']),
                              np.maximum(df['Prev_Low'] - df['Low'], 0), 0)
    
    # 平滑化平均の計算
    df['ATR'] = df['TR'].rolling(window=period).mean()
    df['Plus_DI'] = 100 * (df['Plus_DM'].rolling(window=period).mean() / df['ATR'])
    df['Minus_DI'] = 100 * (df['Minus_DM'].rolling(window=period).mean() / df['ATR'])
    
    # Directional Index (DX) の計算
    df['DX'] = 100 * (abs(df['Plus_DI'] - df['Minus_DI']) / (df['Plus_DI'] + df['Minus_DI']))
    
    # Average Directional Index (ADX) の計算
    df['ADX'] = df['DX'].rolling(window=period).mean()
    
    return df['ADX']

adx_period = 14
data['adx'] = calculate_adx(data, adx_period)

In [4]:
# パラメータの範囲
short_ma_period_range = [5, 10, 15]
long_ma_period_range = [20, 30, 50]
macd_fast_period_range = [8, 12, 16]
macd_slow_period_range = [18, 26, 40]
macd_signal_period_range = [5, 9, 13]
adx_threshold_range = [20, 25, 30]

In [5]:
# 手数料とスリッページ
commission = 0.0005  # 0.05%
slippage = 0.0005    # 0.05%

In [ ]:
best_sharpe = -np.inf
best_params = None
best_data = None

# 全てのパラメータの組み合わせを試す
for short_ma_period, long_ma_period, macd_fast_period, macd_slow_period, macd_signal_period, adx_threshold in product(
    short_ma_period_range, long_ma_period_range, macd_fast_period_range, macd_slow_period_range, macd_signal_period_range, adx_threshold_range):

    # 短期移動平均線が長期より長くならないようにする
    if short_ma_period >= long_ma_period:
        continue
    # MACDの期間設定の妥当性を確認
    if macd_fast_period >= macd_slow_period:
        continue

    df = data.copy()

    # 移動平均線の計算
    df['short_ma'] = df['Close'].rolling(window=short_ma_period).mean()
    df['long_ma'] = df['Close'].rolling(window=long_ma_period).mean()

    # MACDの計算
    df['macd_line'] = df['Close'].ewm(span=macd_fast_period, adjust=False).mean() - df['Close'].ewm(span=macd_slow_period, adjust=False).mean()
    df['signal_line'] = df['macd_line'].ewm(span=macd_signal_period, adjust=False).mean()

    # ADXの計算
    df['adx'] = calculate_adx(df, adx_period)

    # シグナルの生成とポジション管理
    df['Signal'] = 0
    df['Position'] = 0
    position = 0
    entry_price = 0

    for i in range(1, len(df)):
        current = df.iloc[i]
        previous = df.iloc[i - 1]

        # ロングエントリーの条件
        if (current['short_ma'] > current['long_ma']) and (previous['short_ma'] <= previous['long_ma']) and \
           (current['macd_line'] > current['signal_line']) and (previous['macd_line'] <= previous['signal_line']) and \
           (current['adx'] >= adx_threshold) and position == 0:
            df.at[df.index[i], 'Signal'] = 1
            position = 1
            entry_price = current['Close'] * (1 + slippage + commission)

        # ショートエントリーの条件
        elif (current['short_ma'] < current['long_ma']) and (previous['short_ma'] >= previous['long_ma']) and \
             (current['macd_line'] < current['signal_line']) and (previous['macd_line'] >= previous['signal_line']) and \
             (current['adx'] >= adx_threshold) and position == 0:
            df.at[df.index[i], 'Signal'] = -1
            position = -1
            entry_price = current['Close'] * (1 - slippage - commission)

        # ロングポジションのエグジット条件
        elif position == 1:
            if (current['Close'] >= entry_price * 1.02) or (current['Close'] <= entry_price * 0.99) or \
               (current['short_ma'] < current['long_ma']):
                df.at[df.index[i], 'Signal'] = 0
                position = 0

        # ショートポジションのエグジット条件
        elif position == -1:
            if (current['Close'] <= entry_price * 0.98) or (current['Close'] >= entry_price * 1.01) or \
               (current['short_ma'] > current['long_ma']):
                df.at[df.index[i], 'Signal'] = 0
                position = 0

        df.at[df.index[i], 'Position'] = position

    # リターンの計算（手数料とスリッページを考慮）
    df['Return'] = df['Close'].pct_change()
    df['Strategy_Return'] = df['Return'] * df['Position'].shift(1)
    df['Strategy_Return'] -= df['Position'].diff().abs() * (commission + slippage)

    # パフォーマンスの評価
    total_returns = df['Strategy_Return'].sum()
    annual_returns = df['Strategy_Return'].mean() * 252 * 24 * 60
    annual_std = df['Strategy_Return'].std() * np.sqrt(252 * 24 * 60)
    sharpe_ratio = annual_returns / annual_std if annual_std != 0 else 0

    if sharpe_ratio > best_sharpe and total_returns > 0:
        best_sharpe = sharpe_ratio
        best_params = (short_ma_period, long_ma_period, macd_fast_period, macd_slow_period, macd_signal_period, adx_threshold)
        best_data = df.copy()

if best_data is not None:
    print(f"Best Sharpe Ratio: {best_sharpe:.2f}")
    print(f"Best Parameters: Short MA={best_params[0]}, Long MA={best_params[1]}, "
          f"MACD Fast={best_params[2]}, MACD Slow={best_params[3]}, MACD Signal={best_params[4]}, ADX Threshold={best_params[5]}")

    # 累積リターンの計算
    best_data['Cumulative_Strategy_Return'] = (1 + best_data['Strategy_Return']).cumprod()
    best_data['Cumulative_Market_Return'] = (1 + best_data['Return']).cumprod()

    # 結果のプロット
    plt.figure(figsize=(12, 6))
    plt.plot(best_data['Cumulative_Market_Return'], label='Market Return')
    plt.plot(best_data['Cumulative_Strategy_Return'], label='Strategy Return')
    plt.title('Optimized Momentum Strategy Backtest Results')
    plt.xlabel('Time')
    plt.ylabel('Cumulative Returns')
    plt.legend()
    plt.show()
else:
    print("No profitable parameter set found.")